In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

file_path = 'realestate.csv'
df = pd.read_csv(file_path, index_col=False, low_memory=False,)

In [ ]:
# prompt: this is realy big dataset.. i need to make a smaller sample

# Sample 10% of the DataFrame
df = df.sample(frac=0.01, random_state=42) # set random_state for reproducibility

# EDA

In [ ]:
df.head(), df.info()

In [ ]:
df.describe()

In [ ]:
df['property_type'].unique()

In [ ]:
df['residential_type'].unique()

In [ ]:
town_counts = len(df['town'].unique())
town_counts

## Kontrola chybějících hodnot

In [ ]:
df.isnull().sum()

## Kontrola zduplikovaných hodnot

In [ ]:
df.duplicated().sum()

## Datové typy

In [ ]:
df.dtypes

| Název sloupce | Datový typ | Stručný popis |
| --------      | -------    |-------        |
| serial_number | int64   |Unikátní číslo nemovitosti.|
| list_year | int64 | Rok, kdy byla nemovitost zařazena do seznamu.
| date_recorded | object | Datum, kdy byl záznam proveden
| town | object | Název města, kde se nemovitost nachází.
| address | object | Adresa nemovitosti.
|assessed_value | int64 | Odhadovaná hodnota nemovitosti
| sale_amount | float64 |Skutečná částka, za kterou byla nemovitost prodána.
| sales_ratio | float64 | Poměr odhadované hodnoty a skutečné prodejní částky.
| property_type | object | Typ nemovitosti (komerční, rezidenční atd.)
|residential_type | object | Typ rezidenční nemovitosti
| years_until_sold | int64 | Počet let, které uplynuly od zařazení nemovitosti na seznam do jejího prodeje.


## Zjištění unikátních hodnot

In [ ]:
df.nunique()

In [ ]:
# Plot histogram for 'list_year'
plt.figure(figsize=(8, 6))
data = df['list_year'].dropna()  # Remove NaN values
plt.hist(data, bins=len(data.unique()), color='skyblue', edgecolor='black', align='mid')

# Set x-axis ticks to show whole numbers
plt.xticks(sorted(data.unique()), rotation=45)  # Sort to ensure proper order
plt.title('Distribution of List Year', fontsize=14)
plt.xlabel('List Year')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
min_val = -1
max_val = 3

plt.figure(figsize=(8, 6))
data = df['years_until_sold'].dropna()

bins = range(min_val, max_val)
plt.hist(data, bins=bins, color='skyblue', edgecolor='black', align='left')

plt.xticks(range(min_val, max_val), rotation=0)
plt.xlim(min_val, max_val)

plt.title('Distribution of Years Until Sold', fontsize=14)
plt.xlabel('Years Until Sold')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(20, 10))
data = df['town'].dropna()

town_counts = data.value_counts()

town_counts.plot(kind='bar', color='skyblue', edgecolor='black')

plt.title('Frequency of Records by Town', fontsize=14)
plt.xlabel('Town')
plt.ylabel('Frequency')
plt.xticks(rotation=90, ha='right', fontsize=8)

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 8))
property_type_counts = df['property_type'].dropna().value_counts()
property_type_counts.plot(kind='bar', color='skyblue', edgecolor='black')

plt.title('Frequency of Property Types', fontsize=16)
plt.xlabel('Property Type', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 8))
residential_type_counts = df['residential_type'].dropna().value_counts()
residential_type_counts.plot(kind='bar', color='skyblue', edgecolor='black')

plt.title('Frequency of Residential Types', fontsize=16)
plt.xlabel('Residential Type', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
bins = [0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000, float('inf')]
bin_labels = ['0-100k', '100k-200k', '200k-300k', '300k-400k', '400k-500k', '500k-600k',
              '600k-700k', '700k-800k', '800k-900k', '900k-1M', '1M+']

df['sale_amount_range'] = pd.cut(df['sale_amount'], bins=bins, labels=bin_labels, right=False)

plt.figure(figsize=(10,6))
df['sale_amount_range'].value_counts().sort_index().plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Sale Amount Distribution by Range')
plt.xlabel('Sale Amount Range')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()
df = df.drop('sale_amount_range', axis=1)

TODO popsat všechny grafy

In [ ]:
categorical_or_binary = ['assessed_value', 'sale_amount', 'sales_ratio']
selected_columns = [col for col in df.columns if col in categorical_or_binary]
df_only_numeric = df[selected_columns]

fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(15, 15))
axes = axes.flatten()

for i, col in enumerate(df_only_numeric.columns):
    if i < len(axes):  # Check if we still have axes to use
        sns.histplot(df_only_numeric[col], kde=True, ax=axes[i])
        axes[i].set_title(f'Histogram of {col}')
    else:
        break

plt.tight_layout()
plt.show()

# Pre-processing

In [ ]:
# Convert 'date_recorded' to datetime format
df['date_recorded'] = pd.to_datetime(df['date_recorded'], errors='coerce')

# Extract year from the date
df['year_recorded'] = df['date_recorded'].dt.year

In [ ]:
# prompt: drop attribute address
df = df.drop('address', axis=1)

In [ ]:
# One-hot encode the 'property_type' column
df = pd.get_dummies(df, columns=['property_type'], prefix='property_type')

In [ ]:
# One-hot encode the 'property_type' column
df = pd.get_dummies(df, columns=['residential_type'], prefix='residential_type')

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['town'] = label_encoder.fit_transform(df['town'])

In [ ]:
# Filter rows where 'property_type' is 'Single Family'
condo_df = df[df['property_type_Condo'] == True]

# Show the filtered dataframe
condo_df

In [ ]:
filtered_df = condo_df[(condo_df['property_type_Condo'] == True) & (condo_df['residential_type_Condo'] == False)]

filtered_df

Po analýze atributů residential_type a property type jsem se rozhodl smazat atribut residential_type, protože není podstatný pro další analýzu či shlukování. Ověřeno kódem výše. (Toto bylo ověřeno nejen pro typ "Condo" , ale i pro ostatní typy.)

In [ ]:
# prompt: drop all columns where name include residential_type

# Drop columns containing 'residential_type' in their name
df = df.drop(columns=[col for col in df.columns if 'residential_type' in col])

In [ ]:
# prompt: drop date_recorded

df = df.drop('date_recorded', axis=1)

In [ ]:
df = df.drop('serial_number',axis=1)

In [ ]:
# prompt: rename property_type_Nan to other

# Rename the column
df = df.rename(columns={'property_type_Nan': 'property_type_other'})

In [ ]:
# prompt: if it is property_type_Single Family or
# property_type_Three Family or
# property_type_Two Family or
# property_type_Four_Family
# create a new column property_type_Family

# Create 'property_type_Family' column
df['property_type_Family'] = (
    (df['property_type_Single Family'] == 1) |
    (df['property_type_Three Family'] == 1) |
    (df['property_type_Two Family'] == 1) |
    (df['property_type_Four Family'] == 1)
).astype(int)

In [ ]:
# prompt: convert property_type_Family to bool

# The 'property_type_Family' column is already created as an integer (0 or 1)
# representing True or False.  No further conversion is needed.

# If you *really* need a boolean type:
df['property_type_Family'] = df['property_type_Family'].astype(bool)

In [ ]:
# prompt: delete columns: property_type_Single Family, property_type_Three Family, property_type_Two Family, property_type_Four_Family

# Drop the specified columns
df = df.drop(columns=['property_type_Single Family', 'property_type_Three Family', 'property_type_Two Family', 'property_type_Four Family'])

In [ ]:
corr_mat = df.corr()

plt.figure(figsize=(20, 15))
sns.heatmap(corr_mat, cmap='coolwarm', square=True, annot=True, fmt='.2f')

plt.show()

# 1. Metoda shlukování (K-means)

TODO:
list_year, town dummies (one_hot)



assessed_value, sale_amount, sales_ratio - scaler

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df)
    sse.append(kmeans.inertia_)


plt.plot(range(1, 11), sse)
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Sum of Squared Errors (SSE)')
plt.show()

In [ ]:
import sklearn.cluster
clusters = sklearn.cluster.KMeans(n_clusters=2, random_state=13).fit(df)
df['cluster_id'] = clusters.labels_

In [ ]:
# Assuming X_test['cluster_id'] contains the cluster labels
cluster_counts = df['cluster_id'].value_counts()
cluster_counts

## Explorace clusterů

In [ ]:
attributes = ['list_year','assessed_value','sale_amount','town','sales_ratio']

num_attributes = len(attributes)
num_rows = (num_attributes + 1) // 2

fig, axes = plt.subplots(nrows=num_rows, ncols=2, figsize=(12, 4 * num_rows))

axes = axes.flatten()

for i, attribute in enumerate(attributes):
    # Check if the column exists in X_test before plotting
    if attribute in df.columns:
        sns.boxplot(x='cluster_id', y=attribute, data=df, ax=axes[i])
        axes[i].set_title(f'Boxplot of {attribute} by Cluster')
    else:
        print(f"Warning: Column '{attribute}' not found in X_test. Skipping boxplot.")

if num_attributes % 2 != 0:
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

# 2. metoda shlukování (DBSCAN)

# 3. metoda shlukování (Agglomerative Clustering)